In [1]:
# jupyter serverextension enable --sys-prefix voila 

In [2]:
import fastbook
import pandas as pd
import re
import bs4
from fastai.text.all import *
import ipywidgets as widgets

ModuleNotFoundError: No module named 'fastbook'

# Getting Data + Processing

In [ ]:
# def text_processing(x): #cleans up the description text
  
#   x = x.lower()
#   x = bs4.BeautifulSoup(x, "lxml").text
#   x = x.encode('ascii', 'ignore').decode()
#   x = re.sub(r'https*\S+', ' ', x)
#   x = re.sub(r'@\S+', ' ', x)
#   x = re.sub(r'#\S+', ' ', x)
#   x = re.sub(r'\'\w+', '', x)
#   x = re.sub(r'\w*\d+\w*', '', x)
#   x = re.sub(r'\s{2,}', ' ', x)
#   return x 

# def read_df(filename, genre): #reads the data from 3 different files

#   df = pd.read_csv(filename, index_col=0)
#   df.drop(df.index[df['language'] != 'eng'], inplace=True) #keeps only english 
#   df.dropna(subset=['description'], inplace=True) # drops if no description

#   cols_to_keep = ['title', 'author', 'description']
#   df = df[cols_to_keep]

#   df['description'] = df.description.apply(text_processing)
#   df['genre'] = genre

#   return df



In [ ]:
# read data

# data = pd.DataFrame()
# data_genre = {'mystery': 'gr_df_mystery.csv',
#               'sicfi': 'gr_df_sf.csv',
#               'YA': 'gr_df_ya.csv'}

# for genre, filename in data_genre.items():

#   df = read_df(filename, genre)
#   data = pd.concat([data, df])


In [ ]:
# data.tail()

# AI Setup

In [ ]:
# dls = TextDataLoaders.from_df(data, text_col='description', label_col='genre')

In [ ]:
# dls.show_batch(max_n=3)

# We can see that the library automatically processed all the texts to split then in tokens, adding some special tokens like:

#     xxbos to indicate the beginning of a text
#     xxmaj to indicate the next word was capitalized


In [ ]:
# learn = text_classifier_learner(dls, AWD_LSTM, drop_mult=0.5, metrics=accuracy)

In [ ]:
# learn.fine_tune(4, 1e-2)

In [ ]:
# learn.export()

# App

To test, the learner, we provide this sample book description:
<br>
<br>
When Abby signs up for a DNA service, it’s mainly to give her friend and secret love interest, Leo, a nudge. After all, she knows who she is already: Avid photographer. Injury-prone tree climber. Best friend to Leo and Connie…although ever since the B.E.I. (Big Embarrassing Incident) with Leo, things have been awkward on that front. But she didn’t know she’s a younger sister. When the DNA service reveals Abby has a secret sister, shimmery-haired Instagram star Savannah Tully, it’s hard to believe they’re from the same planet, never mind the same parents — especially considering Savannah, queen of green smoothies, is only a year and a half older than Abby herself. The logical course of action? Meet up at summer camp (obviously) and figure out why Abby’s parents gave Savvy up for adoption. But there are complications: Savvy is a rigid rule-follower and total narc. Leo is the camp’s co-chef, putting Abby's growing feelings for him on blast. And her parents have a secret that threatens to unravel everything. But part of life is showing up, leaning in, and learning to fit all your awkward pieces together. Because sometimes, the hardest things can also be the best ones.
<br>
<br>
 From https://www.goodreads.com/book/show/53138158-you-have-a-match

In [ ]:
# Download export.pkl
url = 'https://utexas.box.com/shared/static/igounsdf3z7lp6kednla79o379f4newe.pkl'

import urllib.request

print('Beginning file download with urllib2...')

urllib.request.urlretrieve(url, 'export.pkl')

In [ ]:
def callback(wdgt):
    pred, pred_idx, probs = learn_inf.predict(book_desc.value)
    print(f"Our learner has predicted the books description to be of a {pred} book with probabilty {probs[pred_idx]}")

In [ ]:
#load learner
learn_inf = load_learner('export.pkl')

In [ ]:
#define widget for interactivity
book_desc = widgets.Text(
    value='Type in a book description!',
    placeholder='Type something',
    description='Input:',
    disabled=False
)
display(book_desc)
book_desc.on_submit(callback)